In [60]:
from imp import reload

import numpy as np
import pandas as pd
import sys,math
import sistercellclass as scc; reload(scc)
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]


In [61]:
datadir   = '/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/'
datafiles = [os.path.join(datadir,fn) for fn in os.listdir(datadir) if os.path.splitext(fn)[1][1:].upper() == 'XLS']

#datafiles = ['/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/072818POS9_ch2t_sis2.xls']

data      = scc.SisterCellData(infiles = datafiles, debugmode = False)

In [62]:

for dataID,filename,x in data:
    #print dataID,filename
    cdivA,cdivB = data.CellDivisionTrajectory(dataID)
    #print np.array(cdivA['generationtimeA']),np.array(cdivB['generationtimeB'])
    if dataID == 0:
        gentime = np.concatenate([np.array(cdivA['generationtimeA']),np.array(cdivB['generationtimeB'])])
    else:
        gentime = np.concatenate([gentime,np.array(cdivA['generationtimeA']),np.array(cdivB['generationtimeB'])])
    #print

#np.savetxt('tmp.txt',gentime)
#plt.hist(gentime,range=(0,2),bins=40,log=True)
#print gentime
#print max(gentime)
#gth = dict()
#for x in np.arange(0,2.1,.05):
#    gth['{:.2f}'.format(x)] = 0
#for g in gentime:
#    gth['{:.2f}'.format(g)] += 1

#print gth


/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/101218_Pos8_6_sis.xls B 30
/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/072818_Pos1_7_sis.xls A 63
/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/101218_Pos8_17.xls B 28
/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/072818_Pos4_2_sis.xls B 20
/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/101218_Pos2_14_sis.xls A 112


In [ ]:
print cdivA